# Model Deployment in IBM Cloud

In [21]:
!tar -zcvf fertilizer_disease_prediction.tgz "model.h5"

a model.h5


## Install watson-machine-learning-client module

In [22]:
!pip install ibm-watson-machine-learning


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import the required libraries

In [23]:
from ibm_watson_machine_learning import APIClient

In [24]:
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "qZcRRElMIJl0ACAn7RJRAbfHh05rdUqOQYOre38quL4S"
}

client = APIClient(wml_credentials)

## Store the model in deployment spaces

In [25]:
# function to store the model in deployment space
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (
        next(item for item in space["resources"] if item["entity"]["name"] == space_name)["metadata"]["id"]
    )

In [26]:
space_uid = guid_from_space_name(client, "Deployment Space For Fertilizer Recommendation system")
print("Space UID - " + space_uid)

Space UID - 70c5ee0c-7b1d-492f-9133-180b8cd20d82


## Make the deployment space default

In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

## Choose software specification

In [29]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
print(f"Software Specification UID - {software_spec_uid}")

Software Specification UID - acd9c798-6974-5d2f-a657-ce06e986df4d


In [31]:
model_details = client.repository.store_model(
    model="fertilizer_disease_prediction.tgz",
    meta_props={
        client.repository.ModelMetaNames.NAME: "CNN",
        client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
    }
)
model_id = client.repository.get_model_id(model_details)
print(f"Model ID - {model_id}")

Model ID - fa23ab8e-ef5c-4656-a8f6-e4c7b550e1a6


In [32]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-19T05:57:26.050Z',
  'id': 'fa23ab8e-ef5c-4656-a8f6-e4c7b550e1a6',
  'modified_at': '2022-11-19T06:01:43.556Z',
  'name': 'CNN',
  'owner': 'IBMid-6620043TOR',
  'resource_key': '5bb707b1-5575-457d-b3bf-980b86a3bb57',
  'space_id': '70c5ee0c-7b1d-492f-9133-180b8cd20d82'},
 'system': {'warnings': []}}

In [33]:
# set meta
deployment_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Model Deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [34]:
client.repository.ModelMetaNames.show()

------------------------  ----  --------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME            TYPE  REQUIRED  SCHEMA
NAME                      str   Y
DESCRIPTION               str   N
INPUT_DATA_SCHEMA         list  N         {'id(required)': 'string', 'fields(required)': [{'name(required)': 'string', 'type(required)': 'string', 'nullable(optional)': 'string'}]}
TRAINING_DATA_REFERENCES  list  N         [{'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'endpoint_url(required)': 'string', 'access_key_id(required)': 'string', 'secret_access_key(required)': 'string'},

In [35]:
deployment = client.deployments.create(
    artifact_uid=model_id,
    meta_props=deployment_props
)



#######################################################################################

Synchronous deployment creation for uid: 'fa23ab8e-ef5c-4656-a8f6-e4c7b550e1a6' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='ed9eeb83-13f8-4d42-a66d-dc974c54a0c8'
------------------------------------------------------------------------------------------------




## Download the model

In [36]:
client.repository.download(model_id, filename="fertilizer_recommendation_system_IBM_model.tar.gz")

Successfully saved model content to file: 'fertilizer_recommendation_system_IBM_model.tar.gz'


'C:\\Users\\G.Arunkumar\\.spyder-py3\\Plant-Leaf-Disease-Prediction-main/fertilizer_recommendation_system_IBM_model.tar.gz'